<a href="https://colab.research.google.com/github/harshal306/Chest_Xray_Disease_Detection/blob/master/chest_xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/harshal306/Chest_Xray_Disease_Detection.git

Cloning into 'Chest_Xray_Disease_Detection'...
remote: Enumerating objects: 5882, done.
remote: Counting objects: 100% (5882/5882), done.
remote: Compressing objects: 100% (5869/5869), done.
remote: Total 5882 (delta 20), reused 5862 (delta 10), pack-reused 0
Receiving objects: 100% (5882/5882), 1.15 GiB | 32.67 MiB/s, done.
Resolving deltas: 100% (20/20), done.
Checking out files: 100% (5867/5867), done.


# New Section

In [0]:
!ls

Chest_Xray_Disease_Detection  sample_data


In [0]:
ls

chest_xray/  README.md


In [0]:
pwd

'/content/Chest_Xray_Disease_Detection/chest_xray/train'

In [0]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm

In [0]:
TRAIN_DIR = "/content/Chest_Xray_Disease_Detection/chest_xray/train/data"
TEST_DIR = "/content/Chest_Xray_Disease_Detection/chest_xray/test/data"

In [0]:
IMG_SIZE = 50
LR = 1e-3
MODEL_NAME =('normalvspneumonia-{}-{}.model_train_split'.format(LR,'6conv-basic'))

In [0]:
def label_img(img):
    
#     word_label = img.split('.')[0][:6]
    
    if 'normal' in img : return [1,0]
    
    elif 'pneumonia' in img: return [0,1]


In [0]:
def create_train_data():
    training_data = []
    actual_train_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    actual_train_data = train_data[:int(.8*len(train_data))]
    actual_test_data = train_data[int(.2*len(train_data)):]
    np.save('actual_train_data.npy', actual_train_data)
    
    return (actual_train_data,actual_test_data)


In [0]:
def process_test_data():
    i=0
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        label = label_img(img)
        path = os.path.join(TEST_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), np.array(label)])

    #shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
#process_test_data()

In [246]:
train_data,test_data = create_train_data()
# If you have already created the dataset:
#train_data = np.load('actual_train_data.npy')
#test_data = process_test_data()



  0%|          | 0/5216 [00:00<?, ?it/s]

  0%|          | 14/5216 [00:00<00:41, 125.44it/s]

  0%|          | 22/5216 [00:00<00:48, 107.15it/s]

  1%|          | 32/5216 [00:00<00:51, 100.40it/s]

  1%|          | 41/5216 [00:00<00:54, 95.33it/s] 

  1%|          | 58/5216 [00:00<00:47, 109.22it/s]

  1%|▏         | 68/5216 [00:00<00:51, 100.84it/s]

  2%|▏         | 81/5216 [00:00<00:49, 104.26it/s]

  2%|▏         | 100/5216 [00:00<00:42, 119.92it/s]

  2%|▏         | 113/5216 [00:00<00:43, 117.66it/s]

  2%|▏         | 130/5216 [00:01<00:39, 128.75it/s]

  3%|▎         | 144/5216 [00:01<00:40, 126.52it/s]

  3%|▎         | 158/5216 [00:01<00:41, 122.31it/s]

  3%|▎         | 171/5216 [00:01<00:46, 108.20it/s]

  4%|▎         | 183/5216 [00:01<00:46, 108.27it/s]

  4%|▍         | 198/5216 [00:01<00:42, 117.56it/s]

  4%|▍         | 212/5216 [00:01<00:44, 112.41it/s]

  4%|▍         | 224/5216 [00:01<00:43, 113.71it/s]

  5%|▍         | 236/5216 [00:02<00:44, 111.27it/s]

  5%|▍   

In [0]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

import tensorflow as tf
tf.reset_default_graph()

In [0]:
convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32,2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')

In [0]:
#if os.path.exists('{}.meta'.format(MODEL_NAME)):
    #model.load(MODEL_NAME)
   # print('model loaded!')

In [0]:
train = train_data
test = test_data
#print(train)


In [256]:
X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]
#Y = np.array([i[1] for i in train])
#print(Y)

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]
print(test_y)

[array([0, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([1, 0]), array([0, 1]), array([0, 1]), array([0, 1]), array([0, 1]), array([1,

In [259]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

Training Step: 419  | total loss: 0.04939 | time: 0.764s
| Adam | epoch: 010 | loss: 0.04939 - acc: 0.9853 -- iter: 2624/2669
Training Step: 420  | total loss: 0.04691 | time: 1.786s
| Adam | epoch: 010 | loss: 0.04691 - acc: 0.9852 | val_loss: 1.13098 - val_acc: 0.7869 -- iter: 2669/2669
--


In [0]:
model.save(MODEL_NAME)

In [260]:
import matplotlib.pyplot as plt

# if you need to create the data:
test_data = process_test_data()
# if you already have some saved:
#test_data = np.load('test_data.npy')

fig=plt.figure()
predicted_labels = []

for num,data in enumerate(test_data):
    # normal: [1,0]
    # pneumonia: [0,1]
   
    img_num = data[1]
    img_data = data[0]
    
    #y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    
    if np.argmax(model_out) == 1: 
      str_label='pneumonia'
      inp = np.array([0,1])
      predicted_labels.append(inp)
      
    else: 
      str_label='normal'
      inp = np.array([1,0])
      predicted_labels.append(inp)
        
    #y.imshow(orig,cmap='gray')
    #plt.title(str_label)
    #y.axes.get_xaxis().set_visible(False)
    #y.axes.get_yaxis().set_visible(False)
plt.show()
#print(list(predicted_labels))
#print("OK")
#print(Y)

labels = []
predictions = []
for l,p in zip(test_y,predicted_labels):
  labels.append(l[0])
  predictions.append(p[0])

labels = tf.constant(np.array(labels))
predictions = tf.constant(np.array(predictions))

con_mat = tf.math.confusion_matrix(labels=labels, predictions=predictions, num_classes=2)
with tf.Session():
  print(tf.Tensor.eval(con_mat, feed_dict=None, session=None))



  0%|          | 0/624 [00:00<?, ?it/s]

  3%|▎         | 16/624 [00:00<00:03, 157.69it/s]

  5%|▍         | 29/624 [00:00<00:04, 145.99it/s]

  7%|▋         | 43/624 [00:00<00:04, 143.56it/s]

  8%|▊         | 53/624 [00:00<00:04, 125.93it/s]

 11%|█         | 68/624 [00:00<00:04, 127.73it/s]

 13%|█▎        | 82/624 [00:00<00:04, 131.08it/s]

 15%|█▌        | 94/624 [00:00<00:04, 114.75it/s]

 18%|█▊        | 110/624 [00:00<00:04, 122.68it/s]

 20%|██        | 125/624 [00:00<00:03, 129.42it/s]

 23%|██▎       | 144/624 [00:01<00:03, 140.90it/s]

 25%|██▌       | 159/624 [00:01<00:03, 130.57it/s]

 28%|██▊       | 173/624 [00:01<00:03, 113.40it/s]

 30%|██▉       | 185/624 [00:01<00:04, 103.93it/s]

 31%|███▏      | 196/624 [00:01<00:04, 101.32it/s]

 33%|███▎      | 209/624 [00:01<00:03, 107.55it/s]

 36%|███▌      | 222/624 [00:01<00:03, 112.18it/s]

 38%|███▊      | 238/624 [00:01<00:03, 121.63it/s]

 40%|████      | 251/624 [00:02<00:03, 118.29it/s]

 42%|████▏     | 264/624 [00

<Figure size 432x288 with 0 Axes>

[[380  10]
 [123 111]]
